# PyG to CFGNN

In [1]:
import pickle
from math import ceil

import numpy as np
import torch
from ogb.nodeproppred import PygNodePropPredDataset
from torch.nn.functional import dropout, relu
from torch_geometric.explain import Explainer
from torch_geometric.explain.algorithm import PGExplainer
from torch_geometric.explain.config import ModelConfig
from torch_geometric.explain.metric import fidelity
from torch_geometric.nn import GCNConv
from torch_geometric.utils import k_hop_subgraph, to_dense_adj

np.random.seed(42)
torch.manual_seed(42)

In [2]:
dataset = PygNodePropPredDataset(name="ogbn-arxiv", root="../data/")
graph = dataset[0] # pyg graph object
features = graph.x
edge_index = graph.edge_index
labels = graph.y.flatten()
print(graph)
split_idx = dataset.get_idx_split()
idx_train_sampled = list()
for i in range(40):
    idx = (graph.y[split_idx['train']] == i).nonzero(as_tuple=False)
    idx_sub = np.random.choice(idx[:,0].numpy(), size=10, replace=False)
    idx_train_sampled.extend(idx_sub)

idx_train_full = split_idx['train']
idx_train = torch.tensor(idx_train_sampled, dtype=torch.long)
idx_val = split_idx['valid']
idx_test_full = split_idx['test']

with open("../data/eval-sets/ogbg-arxiv.pickle", "rb") as file:
    eval_indices = pickle.load(file)
eval_indices = sorted([i.item() for i in eval_indices])
idx_test = split_idx['test'][eval_indices]

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])


In [3]:
with open("../data/gnn_explainer/small_amazon.pickle", "rb") as file:
    sa = pickle.load(file)
for key, val in sa.items():
    try:
        print(key, type(val), val.shape)
    except:
        print(key, type(val), len(val))

adj <class 'numpy.ndarray'> (1, 397, 397)
feat <class 'numpy.ndarray'> (1, 397, 745)
labels <class 'numpy.ndarray'> (1, 397)
train_idx <class 'list'> 317
test_idx <class 'list'> 80


In [4]:
cfgnn = dict()
# cfgnn["adj"] = (graph.edge_index, torch.ones(size=(graph.edge_index.size(1),)))
cfgnn["adj"] = to_dense_adj(graph.edge_index).numpy()
cfgnn["feat"] = features.unsqueeze(0).numpy()
cfgnn["labels"] = labels.unsqueeze(0).numpy()
cfgnn["train_idx"] = idx_train.tolist()
cfgnn["test_idx"] = idx_test.tolist()

In [5]:
del graph, features, labels, idx_train, idx_test

In [6]:
for key, val in cfgnn.items():
    try:
        print(key, type(val), val.shape)
    except:
        print(key, type(val), len(val))

adj <class 'numpy.ndarray'> (1, 169343, 169343)
feat <class 'numpy.ndarray'> (1, 169343, 128)
labels <class 'numpy.ndarray'> (1, 169343)
train_idx <class 'list'> 400
test_idx <class 'list'> 1000


In [ ]:
with open("../data/gnn_explainer/ogbg-arxiv.pickle", "wb") as file:
    pickle.dump(cfgnn, file)

: 

: 